# Homework_2022_03_03

<font color=ff0000> 截止日期 4.7 晚上 11:59，4.11日讲评</font>

## 第一部分-要求结果与输出形式完全一致

In [1]:
import pandas as pd
import numpy as np

arrays = [['day', 'day', 'day', 'night', 'night', 'night', 'night', 'night'],
          ['summer', 'winter', 'winter', 'summer', 'summer', 'summer', 'winter', 'winter'],
          ['swim', 'swim', 'soccer', 'swim', 'soccer', 'sleep', 'swim', 'soccer']]

tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['Time', 'Season', 'Sport'])
df1 = pd.DataFrame(np.random.uniform(0, 1, (8, 2)), index=index, columns=['Alice', 'Bob'])

index2 = pd.MultiIndex.from_tuples([('day', 'summer'), ('day', 'winter'), ('night', 'summer'), ('night', 'winter')],
                                   names=['Time', 'Season'])

df2 = pd.DataFrame(list(zip(*[['Awesome', 'Bad', 'Fine', 'Good'], [1, 0, 0.2, 0.4]])), index=index2, columns=['Mood', 'Prob'])

# 不同季节不同时间Alice和Bob去做这些运动的概率
print(df1)
# 不同季节不同时间Alice和Bob的心情，以及该心情对运动概率的影响
print(df2)

                        Alice       Bob
Time  Season Sport                     
day   summer swim    0.167983  0.146023
      winter swim    0.699894  0.516515
             soccer  0.252055  0.131740
night summer swim    0.825215  0.825335
             soccer  0.119466  0.377030
             sleep   0.194170  0.893025
      winter swim    0.170866  0.714395
             soccer  0.214123  0.958579
                 Mood  Prob
Time  Season               
day   summer  Awesome   1.0
      winter      Bad   0.0
night summer     Fine   0.2
      winter     Good   0.4


## 1.1（1分）使用merge得到要求的DataFrame。

In [2]:
# TODO
df_merge = pd.merge(df1, df2, how='outer', left_on=['Time', 'Season'], right_on=['Time', 'Season'], left_index=True, right_index=True)
print(df_merge)

                        Alice       Bob     Mood  Prob
Time  Season Sport                                    
day   summer swim    0.167983  0.146023  Awesome   1.0
      winter swim    0.699894  0.516515      Bad   0.0
             soccer  0.252055  0.131740      Bad   0.0
night summer swim    0.825215  0.825335     Fine   0.2
             soccer  0.119466  0.377030     Fine   0.2
             sleep   0.194170  0.893025     Fine   0.2
      winter swim    0.170866  0.714395     Good   0.4
             soccer  0.214123  0.958579     Good   0.4


## 1.2（1分）计算考虑心情之后的运动概率，直接用原概率乘以Prob。

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html

In [3]:
# TODO
df_mood = df_merge.copy(deep=True)
df_mood['Alice'] = df_merge['Alice']*df_merge['Prob']
df_mood['Bob'] = df_merge['Bob']*df_merge['Prob']
print(df_mood)

                        Alice       Bob     Mood  Prob
Time  Season Sport                                    
day   summer swim    0.167983  0.146023  Awesome   1.0
      winter swim    0.000000  0.000000      Bad   0.0
             soccer  0.000000  0.000000      Bad   0.0
night summer swim    0.165043  0.165067     Fine   0.2
             soccer  0.023893  0.075406     Fine   0.2
             sleep   0.038834  0.178605     Fine   0.2
      winter swim    0.068346  0.285758     Good   0.4
             soccer  0.085649  0.383432     Good   0.4


## 第二部分-不要求结果与输出形式完全一致，合理即可

In [ ]:
# 从word_list_df读出所有的单词，计作word_list，并计算这些单词的最大长度max_length。
import pandas as pd
table = pd.read_excel('poem300.xlsx')
word_list_df = pd.read_excel('wordlist300.xlsx')
word_list = list(word_list_df['word'])
max_length = max(len(w) for w in word_list)
print(len(word_list), word_list)
print(max_length)

In [5]:
print(table)

        ID  Poem_id  line_number             content
0        1     4371         -100  ##餞唐永昌( 一作餞唐郎中洛陽令)
1        2     4371           -1               $$沈佺期
2        3     4371            1      洛陽舊有( 一作出) 神明宰
3        4     4371            2             輦轂由來天地中
4        5     4371            3             餘邑政成何足貴
...    ...      ...          ...                 ...
1796  1797     6651           -1                $$韋丹
1797  1798     6651            1             王事紛紛無暇日
1798  1799     6651            2             浮生冉冉祗如雲
1799  1800     6651            3             已為平子歸休計
1800  1801     6651            4             五老巖前必共聞

[1801 rows x 4 columns]


## 2.1（2分）对poem.xlsx中的诗歌进行向前最大匹配分词，词表使用上一问中的word_list，最大长度使用上一问中的max_length。

In [6]:
# 实现前向最大匹配分词算法
# 要求去除#$ （空格）这三种符号，去除括号内类似（一作 ***）的内容，不必需考虑括号重叠（（））括号缺少）的情况

import re

notation = ['#', '$', ' ']
pattern = '\(.*?\)'
s = 'abcd(ac)cas(sda)'
s = re.sub(pattern, '', s)
print(s)
def cut(string, word_list, max_length):
    # string: 待分词的串
    # word_list: 词表
    # max_length: 词表中词语的最大长度
    # TODO
    st = re.sub(pattern, '', string)
    for nota in notation:
        st = st.replace(nota, '')
    stlist = []
    while len(st)>0:
        if len(st) <= 1:
            stlist.append(st)
            break
        b = False
        for i in range(max_length):
            tmp = st[:max_length-i]
            if tmp in word_list:
                stlist.append(tmp)
                st = st[max_length-i:]
                b = True
                break
        if not b:
            stlist.append(st[:1])
            st = st[1:]
    return stlist
    

table['words'] = table['content'].apply(lambda x: ' '.join(cut(x, word_list, max_length)))
print(table)

abcdcas
        ID  Poem_id  line_number             content          words
0        1     4371         -100  ##餞唐永昌( 一作餞唐郎中洛陽令)         餞 唐 永昌
1        2     4371           -1               $$沈佺期          沈 佺 期
2        3     4371            1      洛陽舊有( 一作出) 神明宰     洛陽 舊有 神明 宰
3        4     4371            2             輦轂由來天地中     輦轂 由來 天地 中
4        5     4371            3             餘邑政成何足貴    餘 邑 政成 何足 貴
...    ...      ...          ...                 ...            ...
1796  1797     6651           -1                $$韋丹            韋 丹
1797  1798     6651            1             王事紛紛無暇日     王事 紛紛 無 暇日
1798  1799     6651            2             浮生冉冉祗如雲     浮生 冉冉 祗 如雲
1799  1800     6651            3             已為平子歸休計  已 為 平 子 歸 休 計
1800  1801     6651            4             五老巖前必共聞    五 老 巖前 必 共聞

[1801 rows x 5 columns]


## 2.2（4分）计算每个词的TF-IDF值。

In [7]:
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table['Poem_id'], split_words, left_index=True, right_on='level_0')


In [157]:
# TF 2分
# 输出形式只是参考，可以和输出形式不一致
# TODO
tf_data = new_data.copy(deep=True)
tf = tf_data.groupby('Poem_id').count().rename(columns={'word':'totcount'})[['totcount']]
tf_data = pd.merge(tf_data, tf, on='Poem_id')
tf2 = tf_data.groupby(['Poem_id', 'word']).count().rename(columns={'level_0':'count'})[['count']]
tf_data = pd.merge(tf_data, tf2, on=['Poem_id', 'word'], how='outer')
tf_data['TF'] = tf_data['count']/tf_data['totcount']
tf_data = tf_data.drop(['totcount', 'count'], axis=1)
tf_data

,Poem_id,level_0,level_1,word,TF
0,4371,0,0,餞,0.040000
1,4371,0,1,唐,0.040000
2,4371,0,2,永昌,0.040000
3,4371,1,0,沈,0.040000
4,4371,1,1,佺,0.040000
...,...,...,...,...,...
6936,6651,1800,0,五,0.033333
6937,6651,1800,1,老,0.033333
6938,6651,1800,2,巖前,0.033333
6939,6651,1800,3,必,0.033333


In [162]:
# IDF 2分
# TODO
N = tf_data['Poem_id'].nunique()
idf_data = tf_data.copy(deep=True)
idf = idf_data.groupby('word').count().rename(columns={'level_1':'IDF'})[['IDF']]
idf['IDF'] = np.log(N/idf['IDF'])
idf_data = pd.merge(idf_data, idf, on='word').sort_values(by=['Poem_id','level_0','level_1'])
idf_data['TF-IDF'] = idf_data['TF']*idf_data['IDF']
idf_data

,Poem_id,level_0,level_1,word,TF,IDF,TF-IDF
0,4371,0,0,餞,0.040000,4.605170,0.184207
3,4371,0,1,唐,0.040000,4.094345,0.163774
8,4371,0,2,永昌,0.040000,4.605170,0.184207
11,4371,1,0,沈,0.040000,4.094345,0.163774
16,4371,1,1,佺,0.040000,5.010635,0.200425
...,...,...,...,...,...,...,...
4560,6651,1800,0,五,0.033333,3.064725,0.102158
5182,6651,1800,1,老,0.033333,3.506558,0.116885
6939,6651,1800,2,巖前,0.033333,5.703782,0.190126
5567,6651,1800,3,必,0.033333,5.010635,0.167021


## 2.3（2分）计算词向量，每一首古诗看作一个向量，向量由对应词的TF-IDF值组成。

In [200]:
# TODO
# 建议先groupby('Poem_id')，然后apply构成向量的函数，最后rename Poem_vec

def fun(x):
    vec = np.zeros(len(word_list))
    for i in range(len(word_list)):
        if word_list[i] in x['word'].unique():
            vec[i] = x.loc[x['word'] == word_list[i]]['TF-IDF'].unique()[0]
    return vec

data = idf_data.copy(deep=True)
poem_vec = data.groupby('Poem_id').apply(fun)

In [207]:
print(poem_vec)
print(poem_vec.iloc[4][4])

Poem_id
4371    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4373    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4394    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4403    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4408    [0.0, 0.0, 0.0, 0.0, 0.27089236388165244, 0.0,...
                              ...                        
6582    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6625    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6626    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6641    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6651    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 300, dtype: object
0.27089236388165244


## 第三部分-选做

## 3.1（1分）（选做1）计算古诗之间的cos similarity表。

In [215]:
# TODO
def similarity(vec):
    matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            x = vec.iloc[i]
            y = vec.iloc[j]
            cosine = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
            matrix[i][j] = cosine
    return matrix

pd.DataFrame(similarity(poem_vec), index=data['Poem_id'].unique(),
             columns=data['Poem_id'].unique())


,4371,4373,4394,4403,4408,4417,4418,4429,4431,4432,...,6462,6463,6559,6579,6581,6582,6625,6626,6641,6651
4371,1.000000,0.062580,0.016082,0.000000,0.000000,0.0,0.044110,0.024045,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4373,0.062580,1.000000,0.008459,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.009408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4394,0.016082,0.008459,1.000000,0.066672,0.000000,0.0,0.018448,0.000000,0.036655,0.0,...,0.000000,0.008631,0.000000,0.000000,0.032250,0.000000,0.000000,0.000000,0.000000,0.000000
4403,0.000000,0.000000,0.066672,1.000000,0.016387,0.0,0.055180,0.000000,0.030114,0.0,...,0.039440,0.000000,0.016643,0.025396,0.037599,0.000000,0.029567,0.000000,0.000000,0.009600
4408,0.000000,0.000000,0.000000,0.016387,1.000000,0.0,0.113920,0.000000,0.000000,0.0,...,0.000000,0.000000,0.032413,0.000000,0.000000,0.019412,0.000000,0.019505,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6582,0.000000,0.000000,0.000000,0.000000,0.019412,0.0,0.000000,0.000000,0.000000,0.0,...,0.012062,0.013581,0.000000,0.066900,0.062649,1.000000,0.015183,0.000000,0.014000,0.013499
6625,0.000000,0.000000,0.000000,0.029567,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.013740,0.000000,0.052256,0.018935,0.015183,1.000000,0.047344,0.014164,0.010239
6626,0.000000,0.000000,0.000000,0.000000,0.019505,0.0,0.017313,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047344,1.000000,0.014573,0.021686
6641,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.012669,0.000000,0.028963,0.000000,0.014000,0.014164,0.014573,1.000000,0.038691


## 3.2（1分）（选做2）实现基于词典的分词方法SPFA。

In [9]:
import queue

class Vertex:
    def __init__(self, key):
        self.id = key
        self.connectedTo = {}

    def add_neighbor(self, nbr, weight=0):
        self.connectedTo[nbr] = weight

    def __str__(self):
        return str(self.id) + ' connectedTo ' + str([x for x in self.connectedTo])

    def get_connections(self):
        return self.connectedTo.keys()

    def get_id(self):
        return self.id

    def get_weight(self, nbr):
        return self.connectedTo[nbr]


class Graph:
    def __init__(self, num=None):
        self.vertList = {}
        self.numVertices = 0
        if num is not None:
            [self.add_vertex(i) for i in range(num)]

    def add_vertex(self, key):
        self.numVertices += 1
        v = Vertex(key)
        self.vertList[key] = v
        return v

    def get_vertex(self, n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self, item):
        return item in self.vertList

    def add_edge(self, u, v, w=float('inf')):
        # u到v的weight
        if u not in self.vertList:
            self.add_vertex(u)
        if v not in self.vertList:
            self.add_vertex(v)
        self.vertList[u].add_neighbor(self.vertList[v].get_id(), w)

    def get_vertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())


def tokenize(sentence, path):
    token = []
    idx = len(path) - 1
    while idx:
        token.append(sentence[path[idx]:idx])
        idx = path[idx]
    token.reverse()
    return token


def spfa(graph):
    # TODO

# 按照PPT的例子，对下面这句话进行分词
# 老师说明天下午休息
sentence = "老师说明天下午休息"
graph = Graph(9)
# 可以理解成左闭右开
graph.add_edge(0, 1, 1)
graph.add_edge(1, 2, 1)
graph.add_edge(2, 3, 1)
graph.add_edge(3, 4, 1)
graph.add_edge(4, 5, 1)
graph.add_edge(5, 6, 1)
graph.add_edge(6, 7, 1)
graph.add_edge(7, 8, 1)
graph.add_edge(8, 9, 1)
graph.add_edge(0, 2, 1)
graph.add_edge(2, 4, 1)
graph.add_edge(4, 6, 1)
graph.add_edge(3, 5, 1)
graph.add_edge(5, 7, 1)
graph.add_edge(7, 9, 1)
path = spfa(graph)
print(path)
print(tokenize(sentence, path))



[0, 0, 0, 2, 2, 3, 4, 5, 7, 7]
['老师', '说', '明天', '下午', '休息']
